# Evaluate

## Tess trained

In [8]:
import sys
from pathlib import Path
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
module_path = str(Path.cwd().parents[0] / "network_models/w2v_emotion_model")
if module_path not in sys.path:
    sys.path.append(module_path)

import utils.wav2Vec_utils as w2vU
import utils.audio_dataset_utils as ADU
import network_models.soundstream_lstm.LSTM_dataset as lds
import network_models.soundstream_lstm.CombinedEmoDataset_7_Emo as ced
import  network_models.w2v_emotion_model.custom_collator as cc
import network_models.w2v_emotion_model.custom_model as cm
import network_models.w2v_emotion_model.model_trainer as ct
import gc

model_name_or_path = "facebook/wav2vec2-large-960h-lv60-self"
pooling_mode = "mean"
device = "cpu"


emo_dataset = ced.CombinedEmoDataSet_7_emos(
    #directory_tess="/home/ckwdani/Music/emotionDatasets/converted_mono/tess",
    directory_cafe="/home/ckwdani/Music/emotionDatasets/converted_mono/cafe",
    #directory_ravdess="/home/ckwdani/Music/emotionDatasets/converted_mono/RAVDESS Audio_Speech_Actors_01-24",
    #                                         directory_mesd="/home/ckwdani/Music/emotionDatasets/converted_mono/mesd",
                                             device=device)

processor, sr = w2vU.init_w2v2(num_labels=len(emo_dataset.label_list), label_list=emo_dataset.label_list, device=device)
newSet = lds.AudioEmotionTessWav2VecDataset(emo_dataset, processor= processor, sampling_rate=sr)

model = cm.Wav2Vec2ForSpeechClassification(model_name_or_path=model_name_or_path, pooling_mode="mean", device=device)
model.freeze_feature_extractor()


In [3]:
model.load_state_dict(torch.load("../content/classifier/W2VClassifier/Nr1/emo_reco_800.pth"))

<All keys matched successfully>

In [9]:
from torch.utils.data import DataLoader
import numpy
import torch.nn.functional as F
trainSet, evalSet = ADU.train_val_dataset(newSet, 0.1)
batch_size = 8
dataset = evalSet
data_collator = cc.DataCollatorCTCWithPadding(processor=processor, padding=True, num_labels=len(newSet.dataSet.label_list))
loader = DataLoader(dataset, shuffle=False, batch_size=batch_size, num_workers=2 ,collate_fn=data_collator.collate_fn)
true, preds = [], []
with torch.no_grad():
    for batch, (X, z) in enumerate(loader):
        X, z = X.to(device),  z.to(device)
        pred = model(X)
        labels = [torch.squeeze(a.nonzero()).item() for a in z]
        true = true + labels
        preds = preds + pred.argmax(1).numpy().tolist()
        gc.collect()
        #preds.append(pred.argmax(1))
        if batch % 10 == 0:
            print(f"{batch*batch_size} of {len(dataset)}")

# correct = 0
# with torch.no_grad():
#     for i in range(len(newSet)):
#         tensor = torch.tensor(newSet[i][0])
#         if(i%10 == 1):
#             print(f"{i} of {len(newSet)}")
#             print(f"correct: {correct*100/i} %")
#         if(model(torch.unsqueeze(tensor, dim=0)).argmax(1) == newSet[i][1][0]):
#             correct = correct + 1
        # print(5 == newSet[500][1][0])
        # print(newSet[500][1][0])
        # print(model(torch.unsqueeze(tensor, dim=0)))
    #print(model(torch.tensor(numpy.expand_dims(newSet[0], axis=0))))

/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/network_models/w2v_emotion_model/custom_model.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = softmax(y)


0 of 94
80 of 94


In [10]:
from sklearn.metrics import classification_report

print(classification_report(true, preds, target_names=emo_dataset.label_list))

              precision    recall  f1-score   support

       angry       0.25      0.36      0.29        14
     disgust       0.00      0.00      0.00        10
        fear       0.12      0.14      0.13        14
       happy       0.13      0.31      0.18        13
     neutral       0.10      0.25      0.14         8
         sad       0.00      0.00      0.00        15
    surprise       0.00      0.00      0.00        20

    accuracy                           0.14        94
   macro avg       0.09      0.15      0.11        94
weighted avg       0.08      0.14      0.10        94



/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicte

In [48]:

tensor = torch.tensor(newSet[324][0])
model(torch.unsqueeze(tensor, dim=0)).argmax(1)

ValueError: `mask_length` has to be smaller than `sequence_length`, but got `mask_length`: 10 and `sequence_length`: 9`

In [10]:
newSet[0][0]

array([ 0.04325287,  0.08506081,  0.06913736, ..., -0.0217979 ,
       -0.0183346 , -0.02307062], dtype=float32)